In [66]:
%matplotlib inline
from matplotlib  import pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict 
import networkx as nx
from tqdm import tqdm

In [67]:
def split2int(s):
    return list(map(int, s.split()))

In [89]:
def end(i):
    return 'e{}'.format(i)
def cache(i):
    return 'c{}'.format(i)

In [246]:
# data = 'me_at_the_zoo.in'
data = 'trending_today.in'
# data = 'videos_worth_spreading.in'
# data = 'kittens.in'
outpath = 'outputs/{}.out'.format(data.split('.')[0])
reqs = {}
end2cache_latency = defaultdict(dict)
CENTER = 'center'
g = nx.Graph()
with open('data/{}'.format(data), 'r') as f:
    V, E, R, C, X = split2int(f.readline())
    video_sizes = {i: int(size)
                   for i, size in enumerate(f.readline().strip().split())}
    for endpoint_i in range(E):
        latency2center, n_caches = split2int(f.readline())
        end2cache_latency[endpoint_i][CENTER] = latency2center                    
        # g.add_edge(end(endpoint_i), CENTER, d=latency2center)
        g.add_node(end(endpoint_i))
        for _ in range(n_caches):
            cache_i, latency = split2int(f.readline())
            end2cache_latency[endpoint_i][cache_i] = latency            
            g.add_edge(end(endpoint_i), cache(cache_i), d=latency)
        endpoint_i += 1
        
    columns = ['vid', 'eid', 'n']
    rows = []
    for l in f:
        rows.append(split2int(l))
    reqs = pd.DataFrame(rows, columns=columns)
for n in g.nodes_iter():
    if n != CENTER:
        g.node[n]['remain'] = X
reqs['vsize'] = reqs['vid'].apply(video_sizes.__getitem__)    

In [159]:
# sort by number of requests (big is good) and size (small is good)
reqs.sort_values(by=['n', 'vsize'], ascending=[False, True], inplace=True)

In [160]:
reqs_array = reqs.as_matrix().tolist()

In [161]:
cache_servers = list(filter(lambda n: n.startswith('c'), g.nodes()))

In [162]:
cache2videos = defaultdict(set)

In [164]:
for n in cache_servers:
    g.node[n]['remain'] = X

# select the request with max number and min size    
ii = 0
while len(reqs_array) > 0:
    ii += 1
    if ii % 10000 == 0:
        print("{} / {}".format(ii, R))
    v, e, n, s = reqs_array.pop(0)
    # if video is already in e, ignore???
    # assign it to the fastest cache server
    able_cache_servers = list(filter(lambda c: g.node[c]['remain'] >= s,
                                     g.neighbors(end(e))))
    if len(able_cache_servers) == 0:  # cache server cannot hold        
        # cache2videos[CENTER].add(v)
        pass
    else:
        best_c = max(able_cache_servers, key=lambda c: end2cache_latency[e][int(c[1:])] - g[end(e)][c]['d'])
        cache2videos[int(best_c[1:])].add(v)
        g.node[best_c]['remain'] -= s    

10000 / 100000
20000 / 100000
30000 / 100000
40000 / 100000
50000 / 100000
60000 / 100000
70000 / 100000
80000 / 100000
90000 / 100000
100000 / 100000


In [165]:
with open(outpath, 'w') as f:
    f.write('{}\n'.format(len(cache2videos)))
    for c, videos in cache2videos.items():
        f.write('{} {}\n'.format(c, ' '.join(map(str, videos))))

In [255]:
vids_by_count = reqs.groupby('vid')['n'].sum().sort_values(ascending=False)

In [256]:
cache_servers = list(filter(lambda n: n.startswith('c'), g.nodes()))
for n in cache_servers:
    g.node[n]['remain'] = X

In [257]:
req_by_videos = {}
for r, sub_df in reqs.groupby('vid'):
    req_by_videos[r] = sub_df

In [258]:
req_by_videos = sorted(req_by_videos.items(), key=lambda t: t[1]['n'].sum(), reverse=True)
req_by_videos = [(r, sdf.as_matrix()) for r, sdf in req_by_videos]

In [268]:
cache2videos = defaultdict(set)
for n in cache_servers:
    g.node[n]['remain'] = X
    
e2c_best = {e: CENTER for e in range(E)}

for v, rs in tqdm(req_by_videos):
    # endpoints that need video i
    # put v into cache that get the max gain
    gain_old = 0
    for i in range(rs.shape[0]):        
        _, e, n, _ = rs[i]
        best_cache = e2c_best[e]
        gain_old += n * (end2cache_latency[e][CENTER] - end2cache_latency[e][best_cache])
    print('gain_old', gain_old)
    
    capable_caches = [c for c in range(C) if g.node[cache(c)]['remain'] >= video_sizes[v]]
    if len(capable_caches) == 0:
        continue
        
    best_gain = gain_old
    best_c = None
    for c in capable_caches:
        gain_new = 0
        for i in range(rs.shape[0]):        
            _, e, n, _ = rs[i]
            if c in end2cache_latency[e]:
                gain_new += n * (end2cache_latency[e][CENTER] - end2cache_latency[e][c])
        if best_gain < gain_new:
            best_gain = gain_new
            best_c = c
        print('gain_new', gain_new)
    # print(best_c)
    if best_c is not None:
        g.node[cache(best_c)]['remain'] -= video_sizes[v]
    #     best_c = int(best_c[1:])
        cache2videos[best_c].add(v)

        for e, c in e2c_best.items():
            if best_c in end2cache_latency[e]:
                if end2cache_latency[e][c] > end2cache_latency[e][best_c]:
                    e2c_best[e] = best_c


  0%|          | 29/10000 [00:00<01:50, 90.24it/s]

gain_old 0
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500
gain_new 71818500


  1%|          | 64/10000 [00:00<01:35, 104.08it/s]

gain_old 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 54272500
gain_new 5


  1%|          | 100/10000 [00:00<01:28, 111.66it/s]

...

 12%|█▏        | 1239/10000 [00:09<00:49, 177.94it/s]

gain_old 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 36311000
gain_new 3

 13%|█▎        | 1277/10000 [00:09<00:49, 175.17it/s]

gain_old 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 35870000
gain_new 3

KeyboardInterrupt: 

In [260]:
with open(outpath, 'w') as f:
    f.write('{}\n'.format(len(cache2videos)))
    for c, videos in cache2videos.items():
        f.write('{} {}\n'.format(c, ' '.join(map(str, videos))))

In [261]:
cache2videos

defaultdict(set, {0: {1051}})